<a href="https://colab.research.google.com/github/FleaBusyBeeBergs/dtsa5506-pipeline/blob/main/dtsa5506-pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt


In [5]:
# load 
data = (pd.read_csv('data/36100434.csv', 
                    low_memory = False)
       )
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287574 entries, 0 to 287573
Data columns (total 17 columns):
 #   Column                                                 Non-Null Count   Dtype  
---  ------                                                 --------------   -----  
 0   REF_DATE                                               287574 non-null  object 
 1   GEO                                                    287574 non-null  object 
 2   DGUID                                                  287574 non-null  object 
 3   Seasonal adjustment                                    287574 non-null  object 
 4   Prices                                                 287574 non-null  object 
 5   North American Industry Classification System (NAICS)  287574 non-null  object 
 6   UOM                                                    287574 non-null  object 
 7   UOM_ID                                                 287574 non-null  int64  
 8   SCALAR_FACTOR                     

In [6]:
data.head()

,REF_DATE,GEO,DGUID,Seasonal adjustment,Prices,North American Industry Classification System (NAICS),UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,1997-01,Canada,2016A000011124,Seasonally adjusted at annual rates,Chained (2017) dollars,All industries [T001],Dollars,81,millions,6,v65201210,1.1.1.1,1204015.0,NaN,NaN,NaN,0
1,1997-01,Canada,2016A000011124,Seasonally adjusted at annual rates,Chained (2017) dollars,Goods-producing industries [T002],Dollars,81,millions,6,v65201211,1.1.1.2,383199.0,NaN,NaN,NaN,0
2,1997-01,Canada,2016A000011124,Seasonally adjusted at annual rates,Chained (2017) dollars,Service-producing industries [T003],Dollars,81,millions,6,v65201212,1.1.1.3,815573.0,NaN,NaN,NaN,0
3,1997-01,Canada,2016A000011124,Seasonally adjusted at annual rates,Chained (2017) dollars,Business sector industries [T004],Dollars,81,millions,6,v65201213,1.1.1.4,NaN,..,NaN,NaN,0
4,1997-01,Canada,2016A000011124,Seasonally adjusted at annual rates,Chained (2017) dollars,"Business sector, goods [T005]",Dollars,81,millions,6,v65201214,1.1.1.5,NaN,..,NaN,NaN,0
